# EPSG コードと JPX コードのデータをマージ

それぞれは `epsg.ipynb`, `jpx.ipynb` で用意した。


In [1]:
import json
import random

## データの読み込み


In [2]:
with open("./output/epsg.json") as f:
    epsg = json.load(f)
len(epsg), epsg

(6934,
 {'10150': {'name': 'MSL UK & Ireland VORF08 depth',
   'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
   'coordinate_operation': None,
   'datum': 'Mean Sea Level UK & Ireland VORF08'},
  '10151': {'name': 'CD UK & Ireland VORF08 depth',
   'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
   'coordinate_operation': None,
   'datum': 'Chart Datum UK & Ireland VORF08'},
  '10156': {'name': 'ETRS89 + MSL UK & Ireland VORF08 depth',
   'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
   'coordinate_operation': None,
   'datum': 'European Terrestrial Reference System 1989 ensemble'},
  '10157': {'name': 'ETRS89 + CD UK & Ireland VORF08 depth',
   'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshor

In [3]:
with open("./output/jpx.json") as f:
    jpx = json.load(f)
len(jpx), jpx

(4360,
 {'1301': {'name': '極洋', 'market': 'プライム（内国株式）', 'industry': '水産・農林業'},
  '1305': {'name': 'iFreeETF TOPIX(年1回決算型)',
   'market': 'ETF・ETN',
   'industry': '-'},
  '1306': {'name': 'NEXT FUNDS TOPIX連動型上場投信',
   'market': 'ETF・ETN',
   'industry': '-'},
  '1308': {'name': '上場インデックスファンドTOPIX', 'market': 'ETF・ETN', 'industry': '-'},
  '1309': {'name': 'NEXT FUNDS ChinaAMC・中国株式・上証50連動型上場投信',
   'market': 'ETF・ETN',
   'industry': '-'},
  '130A': {'name': 'Veritas In Silico',
   'market': 'グロース（内国株式）',
   'industry': '医薬品'},
  '1311': {'name': 'NEXT FUNDS TOPIX Core 30連動型上場投信',
   'market': 'ETF・ETN',
   'industry': '-'},
  '1319': {'name': 'NEXT FUNDS 日経300株価指数連動型上場投信',
   'market': 'ETF・ETN',
   'industry': '-'},
  '131A': {'name': 'CCNグループ', 'market': 'PRO Market', 'industry': '情報・通信業'},
  '1320': {'name': 'iFreeETF 日経225(年1回決算型)',
   'market': 'ETF・ETN',
   'industry': '-'},
  '1321': {'name': 'NEXT FUNDS 日経225連動型上場投信',
   'market': 'ETF・ETN',
   'industry': '-'},
  '1322': {'nam

## 統合


In [4]:
all_codes = {}

for code in set(epsg.keys()) | set(jpx.keys()):
    all_codes[code] = {
        "epsg": epsg.get(code),
        "jpx": jpx.get(code),
    }

len(all_codes)

8672

In [23]:
all_codes = {}

for code in sorted(set(epsg.keys()) | set(jpx.keys())):
    all_codes[code] = {
        "epsg": epsg.get(code),
        "jpx": jpx.get(code),
    }

len(all_codes)

8672

「EPSG・JPX どちらにも存在」「EPSG のみ」「JPX のみ」の 3 パターンでまとめる:


In [24]:
all_codes_grouped = {"both": [], "epsg": [], "jpx": []}

for code, data in all_codes.items():
    if data["epsg"] and data["jpx"]:
        group = "both"
    elif data["epsg"]:
        group = "epsg"
    elif data["jpx"]:
        group = "jpx"

    all_codes_grouped[group].append(
        {
            "code": code,
            **data,
        }
    )

for k, v in all_codes_grouped.items():
    print(k, len(v))

both 2622
epsg 4312
jpx 1738


In [25]:
random.choice(all_codes_grouped["both"])

{'code': '3521',
 'epsg': {'name': 'NAD83(NSRS2007) / Georgia West (ftUS)',
  'area_of_use': 'United States (USA) - Georgia - counties of Baker; Banks; Barrow; Bartow; Ben Hill; Berrien; Bibb; Bleckley; Brooks; Butts; Calhoun; Carroll; Catoosa; Chattahoochee; Chattooga; Cherokee; Clarke; Clay; Clayton; Cobb; Colquitt; Cook; Coweta; Crawford; Crisp; Dade; Dawson; De Kalb; Decatur; Dooly; Dougherty; Douglas; Early; Fannin; Fayette; Floyd; Forsyth; Fulton; Gilmer; Gordon; Grady; Gwinnett; Habersham; Hall; Haralson; Harris; Heard; Henry; Houston; Irwin; Jackson; Jasper; Jones; Lamar; Lee; Lowndes; Lumpkin; Macon; Marion; Meriwether; Miller; Mitchell; Monroe; Morgan; Murray; Muscogee; Newton; Oconee; Paulding; Peach; Pickens; Pike; Polk; Pulaski; Putnam; Quitman; Rabun; Randolph; Rockdale; Schley; Seminole; Spalding; Stewart; Sumter; Talbot; Taylor; Terrell; Thomas; Tift; Towns; Troup; Turner; Twiggs; Union; Upson; Walker; Walton; Webster; White; Whitfield; Wilcox; Worth.',
  'coordinate_op

In [26]:
random.choice(all_codes_grouped["epsg"])

{'code': '7295',
 'epsg': {'name': 'NAD83(2011) / InGCS Fayette-Franklin-Union (m)',
  'area_of_use': 'United States (USA) - Indiana - counties of Fayette, Franklin and Union.',
  'coordinate_operation': 'InGCS Fayette-Franklin-Union (m)',
  'datum': 'NAD83 (National Spatial Reference System 2011)'},
 'jpx': None}

In [27]:
random.choice(all_codes_grouped["jpx"])

{'code': '6037',
 'epsg': None,
 'jpx': {'name': 'ファーストロジック', 'market': 'スタンダード（内国株式）', 'industry': 'サービス業'}}

## 出力


In [28]:
with open("./output/all.json", "w") as f:
    json.dump(all_codes_grouped, f, ensure_ascii=False, indent=4)

In [29]:
!head -20 ./output/all.json

{
    "both": [
        {
            "code": "2001",
            "epsg": {
                "name": "Antigua 1943 / British West Indies Grid",
                "area_of_use": "Antigua island - onshore.",
                "coordinate_operation": "British West Indies Grid",
                "datum": "Antigua 1943"
            },
            "jpx": {
                "name": "ニップン",
                "market": "プライム（内国株式）",
                "industry": "食料品"
            }
        },
        {
            "code": "2002",
            "epsg": {
                "name": "Dominica 1945 / British West Indies Grid",


In [31]:
!ls -alh ./output/all.json

-rw-r--r--  1 sorami  staff   3.7M May 21 19:56 ./output/all.json
